In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

C:\Users\alexi\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\alexi\.cache\huggingface\hub\models--ProsusAI--finbert. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [30]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd

dataset = pd.read_csv("bitcoin_sentiments_21_24_cleaned.csv")

data_text = dataset['Cleaned_Description']
labels = dataset['Sentiment_Category']

In [31]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

preds = []
for text in data_text:
    inputs = tokenizer(text, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = F.softmax(logits, dim=-1)
        label = torch.argmax(probs, dim=1).item()
        preds.append(label)

In [32]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

labels = list(labels)

accuracy = accuracy_score(labels, preds)

precision = precision_score(labels, preds, average='macro')
recall = recall_score(labels, preds, average='macro')
f1 = f1_score(labels, preds, average='macro')

print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 score:  {f1:.4f}")

Accuracy:  0.7424
Precision: 0.7406
Recall:    0.7459
F1 score:  0.7402


In [34]:
dataset

,Date,Cleaned_Description,Sentiment_Category
0,2021-11-05 04:42:00,Bitcoin price is consolidating near the USD 62...,0
1,2021-11-05 08:15:00,Congress could finally approve or reject the m...,2
2,2021-11-05 10:24:00,Bitcoin increasingly becoming a political inst...,2
3,2021-11-05 16:58:00,There is still potential for the price of bitc...,0
4,2021-11-05 21:00:00,Several companies are looking to Latin America...,2
...,...,...,...
11000,2024-09-12 00:00:00,The government of El Salvador has been buying ...,2
11001,2024-09-12 00:00:00,According to data from mempoolspace transactio...,0
11002,2024-09-12 00:00:00,Time Magazine reporter Vera Bergengruen believ...,2
11003,2024-09-12 00:00:00,in bitcoin is concentrated at around 58500 acc...,2


In [35]:
dataset['Benchmark_Predictions'] = preds
dataset

,Date,Cleaned_Description,Sentiment_Category,Benchmark_Predictions
0,2021-11-05 04:42:00,Bitcoin price is consolidating near the USD 62...,0,1
1,2021-11-05 08:15:00,Congress could finally approve or reject the m...,2,1
2,2021-11-05 10:24:00,Bitcoin increasingly becoming a political inst...,2,2
3,2021-11-05 16:58:00,There is still potential for the price of bitc...,0,0
4,2021-11-05 21:00:00,Several companies are looking to Latin America...,2,2
...,...,...,...,...
11000,2024-09-12 00:00:00,The government of El Salvador has been buying ...,2,2
11001,2024-09-12 00:00:00,According to data from mempoolspace transactio...,0,0
11002,2024-09-12 00:00:00,Time Magazine reporter Vera Bergengruen believ...,2,2
11003,2024-09-12 00:00:00,in bitcoin is concentrated at around 58500 acc...,2,2


In [56]:
#dataset.to_csv("bitcoin_sentiments_21_24_cleaned_benchmarked.csv", index=False)

In [6]:
########################################################################################################################################################

##############################################              FINE TUNING               ################################################################

########################################################################################################################################################
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
import pandas as pd

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [7]:
dataset = pd.read_csv("bitcoin_sentiments_21_24_cleaned.csv")
dataset.reset_index(inplace=True)
texts = list(dataset['Cleaned_Description'])
labels = list(dataset['Sentiment_Category'])
dates = list(dataset['Date'])
indexes = list(dataset['index'])

encodings = tokenizer(texts, truncation=True, padding="max_length", max_length=64)

In [8]:
dataset

,index,Date,Cleaned_Description,Sentiment_Category
0,0,2021-11-05 04:42:00,Bitcoin price is consolidating near the USD 62...,0
1,1,2021-11-05 08:15:00,Congress could finally approve or reject the m...,2
2,2,2021-11-05 10:24:00,Bitcoin increasingly becoming a political inst...,2
3,3,2021-11-05 16:58:00,There is still potential for the price of bitc...,0
4,4,2021-11-05 21:00:00,Several companies are looking to Latin America...,2
...,...,...,...,...
11000,11000,2024-09-12 00:00:00,The government of El Salvador has been buying ...,2
11001,11001,2024-09-12 00:00:00,According to data from mempoolspace transactio...,0
11002,11002,2024-09-12 00:00:00,Time Magazine reporter Vera Bergengruen believ...,2
11003,11003,2024-09-12 00:00:00,in bitcoin is concentrated at around 58500 acc...,2


In [9]:
dataset['Sentiment_Category'].value_counts()

Sentiment_Category
2    4292
0    3873
1    2840
Name: count, dtype: int64

In [10]:
from torch.utils.data import Dataset, DataLoader, random_split

class SentimentDataset(Dataset):
    def __init__(self, encodings, labels, indexes, dates, texts):
        self.encodings = encodings
        self.labels = labels
        self.indexes = indexes
        self.dates = dates
        self.texts = texts
    
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        item['indexes'] = self.indexes[idx]
        item['dates'] = self.dates[idx]
        item['texts'] = self.texts[idx]
        return item

dataset = SentimentDataset(encodings, labels, indexes, dates, texts)

In [11]:
train_size = int(0.1 * len(dataset))  # 80% for training
val_size = int(0.1 * len(dataset))    # 10% for validation
test_size = len(dataset) - train_size - val_size  # Remaining 10% for testing

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(eval_pred):
    
    logits, labels = eval_pred
    
    # Convert logits to predicted class IDs
    preds = logits.argmax(axis=-1)
    
    # Compute metrics
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='macro')
    recall = recall_score(labels, preds, average='macro')
    f1 = f1_score(labels, preds, average='macro')

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }


In [13]:
training_args = TrainingArguments(
    output_dir='bitcoin_finbert',
    # num_train_epochs=3,
    max_steps = 400, # comment out this line if you want to train in epochs
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="steps",
    save_strategy="steps",
    warmup_steps = 0.03,
    logging_steps=25,
    eval_steps=25, # comment out this line if you want to evaluate at the end of each epoch
    learning_rate=1e-5,
    save_steps=50,
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

C:\Users\alexi\anaconda3\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
25,0.663100,0.702001,0.722727,0.739731,0.734238,0.717543
50,0.578900,0.553220,0.797273,0.796656,0.793860,0.792952
75,0.626800,0.535352,0.795455,0.794541,0.788024,0.790494
100,0.492000,0.557816,0.791818,0.798540,0.774673,0.781576
125,0.566700,0.564915,0.782727,0.792421,0.785330,0.781678
150,0.463300,0.627189,0.763636,0.791032,0.763700,0.762146
175,0.572200,0.510788,0.804545,0.799565,0.808853,0.802503
200,0.635700,0.513602,0.809091,0.807276,0.807701,0.804964
225,0.533400,0.502188,0.814545,0.810260,0.815056,0.811682
250,0.479200,0.507142,0.816364,0.814217,0.815844,0.813612


TrainOutput(global_step=400, training_loss=0.46430304884910584, metrics={'train_runtime': 1779.4378, 'train_samples_per_second': 0.899, 'train_steps_per_second': 0.225, 'total_flos': 52622683545600.0, 'train_loss': 0.46430304884910584, 'epoch': 1.45})

In [14]:
trainer.evaluate()
predictions = trainer.predict(test_dataset)

In [15]:
predictions

PredictionOutput(predictions=array([[ 0.3200121, -2.4324942,  2.0498855],
       [ 1.3802487, -3.023093 ,  1.1148717],
       [-1.0586864, -1.6603936,  3.0248592],
       ...,
       [ 2.46594  , -1.7899275, -1.9294591],
       [-0.9663449, -1.8868587,  3.0947325],
       [-1.5512599,  3.0546632, -1.2382368]], dtype=float32), label_ids=array([0, 2, 2, ..., 0, 2, 1], dtype=int64), metrics={'test_loss': 0.49490949511528015, 'test_accuracy': 0.8190800681431005, 'test_precision': 0.8148964469825765, 'test_recall': 0.818024949183549, 'test_f1': 0.8161798512582582, 'test_runtime': 628.1078, 'test_samples_per_second': 14.018, 'test_steps_per_second': 3.506})

In [16]:
logits = predictions[0]
labels = predictions[1]

preds = logits.argmax(axis=-1)
    
# Compute metrics
accuracy = accuracy_score(labels, preds)
precision = precision_score(labels, preds, average='macro')
recall = recall_score(labels, preds, average='macro')
f1 = f1_score(labels, preds, average='macro')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.8191
Precision: 0.8149
Recall: 0.8180
F1 Score: 0.8162


In [17]:
test_dataset[0]

{'input_ids': tensor([  101,  1996, 12012,  3222,  1997,  1996, 17094,  2056,  1996,  2506,
         23707,  4740,  2006,  3027,  2595,  6011,  2009,  2081,  1996,  2157,
          2655,  2000,  5851,  3027,  2595,  2015,  3617,  7045,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [18]:
dates_list = []
texts_list = []
indexes_list = []

# Iterate through the dataset
for item in test_dataset:  # Replace `dataset` with your dataset variable name
    dates_list.append(item['dates'])
    texts_list.append(item['texts'])
    indexes_list.append(item['indexes'])

In [19]:
result = pd.DataFrame({
    'index': indexes_list,
    'date': dates_list,
    'text': texts_list,
    'true_label': labels,
    'predicted_label': preds
})

In [20]:
result.to_csv('finetuned_finbert_preds_2.csv', index=False)

In [82]:
model.save_pretrained("bitcoin_finbert_safety")
tokenizer.save_pretrained("bitcoin_finbert_safety")

('bitcoin_finbert_safety\\tokenizer_config.json',
 'bitcoin_finbert_safety\\special_tokens_map.json',
 'bitcoin_finbert_safety\\vocab.txt',
 'bitcoin_finbert_safety\\added_tokens.json',
 'bitcoin_finbert_safety\\tokenizer.json')